1. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát éves bontásban norbert2 azonosítójú ügyfél rendeléseinek értékéről!</span>

1. A lista megfelelően jelölve tartalmazza a végösszeget is!

In [ ]:
select case grouping_id(Year(r.REND_DATUM))
            when 1 then 'Összesen'
            else cast(Year(r.REND_DATUM) as nvarchar(4))
        end  as 'Év',
       sum(MENNYISEG*EGYSEGAR) as 'Rendelés értéke'
from Rendeles_tetel rt join Rendeles r on rt.SORSZAM = r.SORSZAM
where r.[LOGIN] = 'norbert2'
group by rollup(Year(r.REND_DATUM))


2. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát szállítási dátumonként, azon belül szállítási módonként az egyes rendelések összmennyiségéről!</span>

1. Csak azokat a termékeket vegyük figyelembe, amelyek mennyiségi egysége db!
2. A listát szűrjük úgy, hogy az csak a részösszeg sorokat és a végösszeget tartalmazza!

```
3. Hány olyan ügyfél van, aki még nem rendelt semmit?
```

1. Csoportosítsuk őket nem szerint, azon belül életkor szerint!
2. A lista tartalmazza a részösszegeket és a végösszeget is!

![](https://moodle.uni-corvinus.hu/theme/image.php/adaptable/atto_h5p/1709548862/icon)

4. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a megrendelt termékek legkisebb és legnagyobb egységáráról szállítási dátum, azon belül szállítási mód szerinti bontásban!</span>

1. A lista csak a 2015 májusi szállításokat tartalmazza!
2. Jelenítsük meg a részösszegeket és a végösszeget is!

5. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk csoportot a termékek listaára alapján a következők szerint:&nbsp;<br></span> <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Az "olcsó" termékek legyenek azok, amelyek listaára 3000 alatt van.<br>&nbsp;A "drága" termékek legyenek az 5000 felettiek, a többi legyen "közepes".</span>

1. Listázzuk az egyes csoportokat, és a csoportokba tartozó termékek darabszámát!
2. A lista jelenítse meg a végösszeget is!

In [ ]:
select iif(grouping_id(case 
            when LISTAAR < 3000 then 'olcsó'
            when LISTAAR > 5000 then 'drága'
            else 'közepes'
       end) = 1, 'Összesen',  (case 
            when LISTAAR < 3000 then 'olcsó'
            when LISTAAR > 5000 then 'drága'
            else 'közepes'
       end)),     
         count(TERMEKKOD)
from Termek
group by rollup( case 
            when LISTAAR < 3000 then 'olcsó'
            when LISTAAR > 5000 then 'drága'
            else 'közepes'
       end)


6. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát az ügyfelek adatairól név szerinti sorrendben.</span>

1. Minden sorban jelenjen meg a sorrend szerint előző, illetve következő ügyfél neve is.
2. Ha nincs előző vagy következő ügyfél, akkor a 'Nincs' jelenjen meg!

In [ ]:
select *,
       -- előző ügyfél,
       lag(NEV, 1, 'Nincs') over(order by NEV),
       -- következő ügyfél
       lead(NEV, 1, 'Nincs') over(order by NEV)
from Ugyfel
order by NEV

7. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk lekérdezést, amely megmutatja, hogy melyik termékkategóriába hány termék tartozik. A lista a kategória nevét és a darabszámot jelenítse meg.</span>

1. A lista ne tartalmazzon duplikált sorokat.
2. A feladatot partíciók segítségével oldjuk meg!

In [ ]:
select tk.KAT_NEV,
       listaar,
       count(t.TERMEKKOD) over(partition by tk.KAT_NEV order by tk.KAT_NEV)
from Termek t join Termekkategoria tk on t.KAT_ID = tk.KAT_ID
-- group by tk.KAT_NEV

8. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelési tételekről. Az egyes rendelési tételeket termékenként soroljuk be 4 osztályba a rendelés mennyisége alapján. Jelenítsük meg ezt az információt is egy új oszlopban, az oszlop neve legyen 'Mennyiségi kategória'.</span>

1. A lista csak a 100 Ft feletti egységárú rendelési tételeket vegye figyelembe!

In [ ]:
-- ntile függvény - csoportba sorolás, megadhatjuk, 
-- hogy hány csoportba sorolja, egyenlő csoportokra bontja 

9. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Listázzuk a termékek kódját, megnevezését, kategóriájának nevét, és listaárát.</span>

1. A listát egészítsük ki két új oszloppal, amelyek a kategória legolcsóbb, illetve legdrágább termékének árát tartalmazzák.
2. A két új oszlop létrehozásánál partíciókkal dolgozzunk!

In [ ]:
-- first_value() és last_value() fv-ek (partíció legelső és legutolsó eleme - legolcsóbb és legdrágább termék)

10. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelésekről. A lista legyen rendezve ügyfelenként (LOGIN), azon belül a rendelés dátuma szerint. A listához készítsünk sorszámozást is. A sorszám a következő formában jelenjen meg: sorszám_év_login. Pl: 1_2015_adam1</span>

1. A számozás login-onként, azon belül rendelési évenként kezdődjön újra.
2. A sorszám oszlop neve legyen Azonosító

11. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a termékek adatairól listaár szerint növekvő sorrendben! A lista jelenítse meg két új oszlopban a sorrend szerint előző, illetve következő termék listaárát is a termék saját kategóriájában és raktárában!</span>

1. Ahol nincs előző vagy következő érték, ott 0 jelenjen meg!
2. Az oszlopokat nevezzük el értelemszerűen!

12. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Listázzuk a termékek kódját, nevét és listaárát listaár szerinti sorrendben!</span>

1. Vegyünk fel egy új oszlopot Mozgóátlag néven, amely minden esetben az aktuális termék, az előző, és a következő termék átlagárát tartalmazza!
2. A mozgóátlagot kerekítsük két tizedesre!

In [ ]:
-- fizikai ablak
select TERMEKKOD,
       MEGNEVEZES, 
       LISTAAR,
       -- mozgóátlag = (előző + aktuális + következő) / 3
       (lag(LISTAAR) over(order by listaar) + 
       LISTAAR + 
       lead(LISTAAR) over(order by listaar)) / 3 as 'Mozgóátlag',
       -- fizikai ablak: 3 soros 
       AVG(LISTAAR) over(order by listaar rows between 1 preceding and 1 following) as 'Mozgóátlag 2'
from Termek
order by LISTAAR

13. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát, amely a rendelések sorszámát és a rendelés értékét tartalmazza. A listát egészítsük ki egy új oszloppal, amely minden rendelés esetén addigi rendelések összegét tartalmazza (az aktuálisat is beleértve)!</span>

1. A listát rendezzük sorszám szerint növekvő sorrendbe.
2. A lista ne tartalmazzon duplikált sorokat!

In [ ]:
-- logikai ablak elejétől az aktuálisig
select distinct SORSZAM,
       -- aktuális rendelés értéke
       sum(MENNYISEG * EGYSEGAR) over(partition by SORSZAM),
       -- eddigi értékek összértéke
       sum(MENNYISEG * EGYSEGAR) over(order by SORSZAM
       range between unbounded preceding and current row)
       as 'Göngyölített összeg'
from Rendeles_tetel
--group by SORSZAM
order by SORSZAM

14. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a termékek kódjáról, nevéről, kategória azonosítójáról, raktár azonosítójáról és listaáráról, valamint a termék adott szempontok szerinti rangsorokban elfoglalt helyezéseiről. (Szempontonként külön oszlopban, a helyezéseknél növekvő sorrendet feltételezve). A szempontok a következők legyenek: listaár, kategória szerinti listaár, és raktárkód szerinti listaár.</span>

1. Az oszlopokat nevezzük el értelemszerűen.
2. A helyezések egyenlőség esetén "sűrűn" kövessék egymást.
3. A lista legyen rendezett kategória azonosító, azon belül listaár szerint!

In [32]:
-- sorszámozás: row_mumber() (rank(), dense_rank())
-- beszámozza az elemeket (pl 1-es termék a legolcsóbb stb --> row_number() - nem számít az azonos érték)
-- rank egyforma számot ad azonos elemeknek, de ha pl két első van, akkor az első három sorszáma: 1, 1, 3
-- dense_rank: uez, mint a rank, de a sorszámok: 1, 1, 2

select -- sorszám
       dense_rank() over(order by listaar) as 'Listaár szerinti sorszám',
       dense_rank() over(order by KAT_ID) as 'Kategória szerinti sorszám',
       dense_rank() over(order by RAKTAR_KOD) as 'Raktár szerinti sorszám',
       TERMEKKOD as 'Termékkód',
       MEGNEVEZES as 'Termék neve',
       KAT_ID as 'Kategória',
       RAKTAR_KOD as 'Raktár',
       LISTAAR as 'Listaár'
from Termek

(551 rows affected)

Total execution time: 00:00:00.016

Listaár szerinti sorszám,Kategória szerinti sorszám,Raktár szerinti sorszám,Termékkód,Termék neve,Kategória,Raktár,Listaár
149,31,1,L023-FE301,lány alsó,35,1,390
206,33,1,F021-IN215,férfi ing,37,1,6490
216,34,1,F001-KB101,férfi szövet kabát,38,1,22000
213,35,1,F013-NA207,férfi szövet nadrág,39,1,11900
207,50,1,G064-CI401,gyermek szandál (fiú),54,1,6900
212,51,1,L001-KB101,gyermek kabát (lány),55,1,8900
212,51,1,G001-KB101,gyermek kabát (fiú),55,1,8900
195,52,1,L019-NA207,gyermek pamut nadrág (lány),56,1,1290
210,50,1,L082-CI419,gyermek csizma (lány),54,1,8000
217,97,1,N001-KB101,női szövet kabát,107,1,24000


15. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelési tételekről, amely minden sor esetén göngyölítve tartalmazza az ügyfél adott rendelési tételig meglévő rendelési tételeinek összértékét!</span>

1. Az új oszlop neve legyen Eddigi rendelési tételek összértéke!
2. Az ügyfél neve is jelenjen meg!